## Initializing Spark Environment and Loading data

In [1]:
import findspark
findspark.init('/home/shahayush954/spark-3.4.1-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp_project').getOrCreate()

23/08/23 23:50:51 WARN Utils: Your hostname, ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/23 23:50:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 23:50:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv('SMSSpamCollection', inferSchema=True, sep='\t') # separated by tabs

In [4]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Adding Length feature

In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length', length(data['text']))

In [10]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



## Relation between Spam and Ham with respect to length

In [11]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



Spam messages have higher mean lengths as compared to Ham messages

## Data Preparation and cleaning

In [12]:
from pyspark.ml.feature import Tokenizer, IDF, StringIndexer, StopWordsRemover, CountVectorizer

In [13]:
tokenizer = Tokenizer(inputCol='text', outputCol='tokens')
stop_remover = StopWordsRemover(inputCol='tokens', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='count_vec')
idf = IDF(inputCol='count_vec', outputCol='tf_idf')
stringIndexer = StringIndexer(inputCol='class', outputCol='label')

In [17]:
tokenized_data = tokenizer.transform(data)t

In [19]:
tokenized_data.show()

+-----+--------------------+------+--------------------+
|class|                text|length|              tokens|
+-----+--------------------+------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|
| spam|FreeMsg Hey there...|   147|[freemsg, hey, th...|
|  ham|Even my brother i...|    77|[even, my, brothe...|
|  ham|As per your reque...|   160|[as, per, your, r...|
| spam|WINNER!! As a val...|   157|[winner!!, as, a,...|
| spam|Had your mobile 1...|   154|[had, your, mobil...|
|  ham|I'm gonna be home...|   109|[i'm, gonna, be, ...|
| spam|SIX chances to wi...|   136|[six, chances, to...|
| spam|URGENT! You have ...|   155|[urgent!, you, ha...|
|  ham|I've been searchi...|   196|[i've, been, sear...|
|  ham|I HAVE A DATE ON ...|   

In [18]:
stop_words_removed_data = stop_remover.transform(tokenized_data)

In [20]:
stop_words_removed_data.show()

+-----+--------------------+------+--------------------+--------------------+
|class|                text|length|              tokens|         stop_tokens|
+-----+--------------------+------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, think, goes...|
| spam|FreeMsg Hey there...|   147|[freemsg, hey, th...|[freemsg, hey, da...|
|  ham|Even my brother i...|    77|[even, my, brothe...|[even, brother, l...|
|  ham|As per your reque...|   160|[as, per, your, r...|[per, request, 'm...|
| spam|WINNER!! As a val...|   157|[winner!!, as, a,...|[winner!!, valued...|
| spam|Had your mobile 1...|   154|[had, your, mobil...|[mobile,

In [21]:
count_vec_data = count_vec.fit(stop_words_removed_data).transform(stop_words_removed_data)
count_vec_data.show()

+-----+--------------------+------+--------------------+--------------------+--------------------+
|class|                text|length|              tokens|         stop_tokens|           count_vec|
+-----+--------------------+------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, think, goes...|(13423,[36,134,31...|
| spam|FreeMsg Hey there...|   147|[freemsg, hey, th...|[freemsg, hey, da...|(13423,[10,60,140...|
|  ham|Even my brother i...|    77|[even, my, brothe...|[even, brother, l...|(13423,[10,53,102...|
|  ham|As 

In [22]:
idf_data = idf.fit(count_vec_data).transform(count_vec_data)
idf_data.show()

+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|              tokens|         stop_tokens|           count_vec|              tf_idf|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, think, goes...|(13423,[36,134,31...|(13423,[36,134,31...|
| spam|FreeMsg Hey there...|   147|[free

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
assembler = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [25]:
features_data = assembler.transform(idf_data)
features_data.show()

+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|              tokens|         stop_tokens|           count_vec|              tf_idf|            features|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|(13424,[0,24,301,...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...|(13424,[0,70,80,1...|
|  ham|Nah I 

In [26]:
features_with_labels = stringIndexer.fit(features_data).transform(features_data)
features_with_labels.show()

+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|class|                text|length|              tokens|         stop_tokens|           count_vec|              tf_idf|            features|label|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|  0.0|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|(13424,[0,24,301,...|  0.0|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|  1.0|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...

In [27]:
from pyspark.ml.classification import NaiveBayes

In [28]:
naiveBayes = NaiveBayes()

In [29]:
train_data, test_data = features_with_labels.randomSplit([0.7,0.3])

In [30]:
model = naiveBayes.fit(train_data)

23/08/24 00:13:51 WARN DAGScheduler: Broadcasting large task binary with size 1197.6 KiB
23/08/24 00:13:54 WARN DAGScheduler: Broadcasting large task binary with size 1148.9 KiB


In [31]:
results = model.transform(test_data)

In [32]:
results.show()

23/08/24 00:15:10 WARN DAGScheduler: Broadcasting large task binary with size 1417.1 KiB


+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|class|                text|length|              tokens|         stop_tokens|           count_vec|              tf_idf|            features|label|       rawPrediction|         probability|prediction|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|  ham| came to look at ...|   103|[, came, to, look...|[, came, look, fl...|(13423,[3,11,163,...|(13423,[3,11,163,...|(13424,[3,11,163,...|  0.0|[-1031.0617756918...|[1.0,5.5967586141...|       0.0|
|  ham| says that he's q...|   200|[, says, that, he...|[, says, quitting...|(13423,[0,3,14,18...|(13423,[0,3,14,18...|(13424,[0,3,14,18...|  0.0|[-1200.9152618364...|[1.0,7.8517820042...|       0.0|


In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
evaluator = MulticlassClassificationEvaluator()

In [35]:
accuracy = evaluator.evaluate(results)

23/08/24 00:16:24 WARN DAGScheduler: Broadcasting large task binary with size 1407.6 KiB


In [36]:
accuracy

0.9274953927370526